# Sample run for Fisher Market

In [1]:
using Pkg
Pkg.activate("../")

  Activating project at `~/workspace/ExchangeMarket.jl/scripts`


In [2]:
using Revise
using SparseArrays, LinearAlgebra
using JuMP, MosekTools
using Plots, LaTeXStrings, Printf, Random
import MathOptInterface as MOI

using ExchangeMarket

include("../tools.jl")
include("../plots.jl")
include("setup.jl")
switch_to_pdf(;)


:pdf

## Test a CES Economy

Run CES economy by HessianBarrier method (primal-dual predictor-corrector mode)

- note this example is `not` so small, use smaller $n, m$ if needed

In [3]:
Random.seed!(1234)
n = 50
m = 1000

1000

In [4]:
method = HessianBar
kwargs = Dict(
    :tol => 1e-10,
    :maxiter => 20,
    :option_step => :logbar,
    :linsys => :DRq,
)


f0 = FisherMarket(m, n; ρ=0.8, bool_unit=true, sparsity=0.95, bool_force_dense=true)
linconstr = LinearConstr(1, n, ones(1, n), [sum(f0.w)])

f1 = copy(f0)
p₀ = ones(n) ./ m
x₀ = ones(n, m) / m
f1.x .= x₀
f1.p .= p₀

alg = method(
    n, m, p₀;
    optimizer=CESAnalytic,
    linconstr=linconstr,
    kwargs...
)

FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.09089493751525879 seconds
FisherMarket initialized in 0.09203577041625977 seconds
FisherMarket initialization started...
FisherMarket cost matrix initialized in 0.00537109375 seconds
FisherMarket initialized in 0.009090900421142578 seconds


Dict{Int64, Vector{Rational{Int64}}} with 2 entries:
  2 => [501, 502, 503, 504, 505, 506, 507, 508, 509, 510  …  991, 992, 993, 994…
  1 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  491, 492, 493, 494, 495, 496, 497, 49…

### Define a cluster map

In [7]:
alg.Ha.cluster_map = Dict(1 => [1:m//2...], 2 => [m//2+1:m...])
alg.Ha.cluster_map

Dict{Int64, Vector{Int64}} with 2 entries:
  2 => [501, 502, 503, 504, 505, 506, 507, 508, 509, 510  …  991, 992, 993, 994…
  1 => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  491, 492, 493, 494, 495, 496, 497, 49…

In [6]:
traj = opt!(
    alg, f1;
    keep_traj=true,
    tol=1e-8
);

--------------------------------------------------------------------------------------------
                   ExchangeMarket.jl: A Julia Package for Exchange Market                   
                                    © Chuwen Zhang (2024)                                    
--------------------------------------------------------------------------------------------
 subproblem solver alias       := CESAnalytic
 subproblem solver style       := analytic
 lin-system solver alias       := DRq
 option for gradient           := dual
 option for step               := logbar
 option for μ                  := normal
--------------------------------------------------------------------------------------------
      k |  lg(μ) |             φ |    |∇φ| |    |Δp| |       t |      tₗ |       α |     kᵢ 
      0 |  -6.00 | +9.486221e-01 | 1.4e+02 | 1.3e-01 | 1.0e+00 | 4.0e-01 | 1.0e+00 | 2.0e+01 
      1 |  -7.00 | -1.091661e+00 | 1.8e+00 | 8.7e-03 | 1.1e+00 | 4.0e-01 | 1.0e+00 | 2.0e+01 
     

### Validate results

In [ ]:
validate(f1, alg)

### Plot trajectory 

we plot the dual local norm $\|\nabla \varphi\|_{\mathbf{p}}^*$ of the trajectory.

In [ ]:
fig = generate_empty(; shape=:wide)
plot!(
    ylabel=L"$\|\nabla \varphi\|_{\mathbf{p}}^*$",
    legendbackgroundcolor=RGBA(1.0, 1.0, 1.0, 0.8),
    yticks=10.0 .^ (-10:3:3),
)

ρfmt = @sprintf("%+.2f", f1.ρ)
σfmt = @sprintf("%+.2f", f1.σ)
traj_pp₊ = map(pp -> pp.gₙ + 1e-18, traj)
plot!(fig, traj_pp₊[2:end], label=L"$\rho := %$ρfmt~(\sigma_i := %$σfmt)$", linewidth=2, markershape=:circle)

fig